<a href="https://colab.research.google.com/github/fdolucas6/IRIS-regression-model-and-data-visualization-/blob/main/Food%20Delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FOOD DELIVERY TIME PREDICTION

In [ ]:
import pandas as pd

# Load the provided dataset
file_path = '//content/train.csv'
data = pd.read_csv(file_path)

# Display basic information about the dataset
print(data.info())
#print(data.head())

# Data cleaning and preprocessing

# Clean the target variable
data['Time_taken(min)'] = data['Time_taken(min)'].str.extract('(\d+)').astype(float)
print(data['Time_taken(min)'].head())

# Convert relevant columns to numeric
data['Delivery_person_Age'] = pd.to_numeric(data['Delivery_person_Age'], errors='coerce')
data['Delivery_person_Ratings'] = pd.to_numeric(data['Delivery_person_Ratings'], errors='coerce')
data['multiple_deliveries'] = pd.to_numeric(data['multiple_deliveries'], errors='coerce')

# Convert dates and times to datetime
data['Order_Date'] = pd.to_datetime(data['Order_Date'], format='%d-%m-%Y', errors='coerce')
data['Time_Orderd'] = pd.to_datetime(data['Time_Orderd'], format='%H:%M:%S', errors='coerce')
data['Time_Order_picked'] = pd.to_datetime(data['Time_Order_picked'], format='%H:%M:%S', errors='coerce')

# display  data after changed
#print("data print after convet to numeric", data.head())


#Check for missing values after conversion
missing_values = data.isnull().sum()

# Display cleaned data overview and missing values
print(data.info(), missing_values)

# Calculate distance between restaurant and delivery location
from geopy.distance import geodesic
data['Distance'] = data.apply(
    lambda row: geodesic(
        (row['Restaurant_latitude'], row['Restaurant_longitude']),
        (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    ).km, axis=1
)
print("data print after calculate distance", data['Distance'].head())


# Drop rows with missing time data as they are crucial for feature engineering
data = data.dropna(subset=['Time_Orderd','Delivery_person_Age','Delivery_person_Ratings','multiple_deliveries'])
print("data print after drop missing value", data.head())

missing_values = data.isnull().sum()
print(data.info(), missing_values)

# Calculate preparation time in minutes
data['Preparation_Time'] = (data['Time_Order_picked'] - data['Time_Orderd']).dt.total_seconds() / 60
print("data print after calculate preparation time", data['Preparation_Time'].head())

from sklearn import preprocessing
le_color = preprocessing.LabelEncoder()
data["Weatherconditions"] = le_color.fit_transform(data["Weatherconditions"])
data["Road_traffic_density"] = le_color.fit_transform(data["Road_traffic_density"])
data["Type_of_order"] = le_color.fit_transform(data["Type_of_order"])
data["Type_of_vehicle"] = le_color.fit_transform(data["Type_of_vehicle"])
data["City"] = le_color.fit_transform(data["City"])
data["Festival"] = le_color.fit_transform(data["Festival"])

print("data print after label encoding", data.head())


# Define features and target
X = data.drop(columns=['ID', 'Delivery_person_ID', 'Time_taken(min)', 'Order_Date',
                                'Time_Orderd', 'Time_Order_picked'])
y = data['Time_taken(min)']
print ("data after define feature and target", X.head())
print ("data after define feature and target", y.head())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_scalar=scaler.transform(X)
df=pd.DataFrame(X_scalar)
print(df.head(10))

from sklearn.model_selection import train_test_split
# Dropping any rows with Nan values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,shuffle=True)
from sklearn.linear_model import LinearRegression
# Splitting the data into training and testing data
regr = LinearRegression()
regr.fit(X_train, y_train)
res=regr.predict(X_test)
print(regr.score(X_test, y_test))


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
model = RandomForestRegressor(random_state=50)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, r2



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

<ipython-input-2-53f990d6bc7a>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Preparation_Time'] = (data['Time_Order_picked'] - data['Time_Orderd']).dt.total_seconds() / 60


data print after label encoding         ID Delivery_person_ID  Delivery_person_Age  Delivery_person_Ratings  \
0  0x4607     INDORES13DEL02                  37.0                      4.9   
1  0xb379     BANGRES18DEL02                  34.0                      4.5   
2  0x5d6d     BANGRES19DEL01                  23.0                      4.4   
3  0x7a6a    COIMBRES13DEL02                  38.0                      4.7   
4  0x70a2     CHENRES12DEL01                  32.0                      4.6   

   Restaurant_latitude  Restaurant_longitude  Delivery_location_latitude  \
0            22.745049             75.892471                   22.765049   
1            12.913041             77.683237                   13.043041   
2            12.914264             77.678400                   12.924264   
3            11.003669             76.976494                   11.053669   
4            12.972793             80.249982                   13.012793   

   Delivery_location_longitude Order

(3.034660728934724, 0.8319906203400285)